In [1]:
import pandas as pd
import pareto
import os
import sys
from itertools import chain
from matplotlib import pyplot as plt

sys.path.append("..")

In [2]:
import rbf_functions

# Load solutions for each RBF for all seeds:

In [4]:
import os

rbfs = [
    rbf_functions.squared_exponential_rbf,
    rbf_functions.original_rbf,
    rbf_functions.inverse_quadratic_rbf,
    rbf_functions.inverse_multiquadric_rbf,
    rbf_functions.exponential_rbf,
    rbf_functions.matern32_rbf,
    rbf_functions.matern52_rbf,
]

root_dir = "../output//"
pareto_sets = {}
for entry in rbfs:
    sollist = []
    solutions = []
    name = entry.__name__
    output_dir = os.path.join(root_dir, name)

    for filename in os.listdir(output_dir):
        if filename.endswith("solution.csv"):
            sollist.append(filename[:-4])
            df_temp = pd.read_csv(f"{output_dir}/{filename}", header=0)
            solutions.append(df_temp.values.tolist())
    pareto_sets[name] = list(chain.from_iterable(solutions))

pareto_sets.keys()
print("Amount of solutions for each RBF:")
for rbf in pareto_sets:
    print(rbf, len(pareto_sets[rbf]))
# print(f"Loaded into list 'solutions': {', '.join(sollist)}")

Amount of solutions for each RBF:
squared_exponential_rbf 213
original_rbf 186
inverse_quadratic_rbf 177
inverse_multiquadric_rbf 179
exponential_rbf 186
matern32_rbf 184
matern52_rbf 191


# Create reference set for each RBF, and save:

In [5]:
reference_sets = {}
for rbf in pareto_sets:
    print(rbf, len(pareto_sets[rbf]))
    nondominated = pareto.eps_sort(
        [pareto_sets[rbf]], [0, 1, 2, 3], [0.05, 0.05, 0.05, 0.05], maximize=[1, 2, 3]
    )
    reference_sets[rbf] = nondominated
    df_nondom = pd.DataFrame(
        nondominated, columns=["max_p", "utility", "inertia", "reliability"]
    )
    print(rbf, len(df_nondom))
    df_nondom.to_csv(f"./{rbf}_refset.csv", index=False, header=True)

squared_exponential_rbf 213
squared_exponential_rbf 23
original_rbf 186
original_rbf 21
inverse_quadratic_rbf 177
inverse_quadratic_rbf 19
inverse_multiquadric_rbf 179
inverse_multiquadric_rbf 19
exponential_rbf 186
exponential_rbf 23
matern32_rbf 184
matern32_rbf 20
matern52_rbf 191
matern52_rbf 22


# Find decision variables that belong to the generated refset:

In [6]:
d_refvar = {}
d_refsol = {}
for entry in rbfs:
    name = entry.__name__
    output_dir = os.path.join(root_dir, name)

    ts = []
    tv = []
    for filename in os.listdir(output_dir):
        # load
        if filename.endswith("solution.csv"):
            df_ts = pd.read_csv(f"{output_dir}/{filename}", header=0)
            ts.append(df_ts)
        elif filename.endswith("variables.csv"):
            df_tv = pd.read_csv(f"{output_dir}/{filename}", header=None)
            tv.append(df_tv)

    df_sol = pd.concat(ts)
    df_var = pd.concat(tv)

    sol = df_sol.values.tolist()
    var = df_var.values.tolist()
    ref = reference_sets[name]
    refvar = []
    refsol = []
    for idx, value in enumerate(sol):
        if value in ref:
            refvar.append(var[idx])
            refsol.append(sol[idx])
    d_refsol[name] = pd.DataFrame(
        refsol, columns=["max_p", "utility", "inertia", "reliability"]
    )
    d_refvar[name] = pd.DataFrame(refvar)
    d_refsol[name].to_csv(f"./refsets/{name}_refset.csv", index=False, header=True)
    d_refvar[name].to_csv(
        f"./refsets/{name}_refset_variables.csv", index=False, header=False
    )

In [7]:
for entry in rbfs:
    name = entry.__name__
    print(name)
    print(f"refset: {len(d_refsol[name])}")
    print(f"varset: {len(d_refvar[name])}")

squared_exponential_rbf
refset: 23
varset: 23
original_rbf
refset: 21
varset: 21
inverse_quadratic_rbf
refset: 19
varset: 19
inverse_multiquadric_rbf
refset: 19
varset: 19
exponential_rbf
refset: 23
varset: 23
matern32_rbf
refset: 20
varset: 20
matern52_rbf
refset: 22
varset: 22


# Generate global reference set for all RBFs:

In [8]:
x = 0
for rbf in pareto_sets:
    x += len(pareto_sets[rbf])
    print(rbf, len(pareto_sets[rbf]))
print("total:", x)

pareto_set = {}
sollist = []
solutions = []
for entry in rbfs:
    name = entry.__name__
    output_dir = os.path.join(root_dir, name)
    for filename in os.listdir(output_dir):
        if filename.endswith("solution.csv"):
            sollist.append(filename[:-4])
            df_temp = pd.read_csv(f"{output_dir}/{filename}", header=0)
            solutions.append(df_temp.values.tolist())
pareto_set = list(chain.from_iterable(solutions))
len(pareto_set)

squared_exponential_rbf 213
original_rbf 186
inverse_quadratic_rbf 177
inverse_multiquadric_rbf 179
exponential_rbf 186
matern32_rbf 184
matern52_rbf 191
total: 1316


1316

In [9]:
print(len(pareto_set))
nondominated = pareto.eps_sort(
    [pareto_set], [0, 1, 2, 3], [0.05, 0.05, 0.05, 0.05], maximize=[1, 2, 3]
)
df_nondom = pd.DataFrame(
    nondominated, columns=["max_p", "utility", "inertia", "reliability"]
)
print(len(nondominated))
df_nondom.to_csv(f"./refsets/all_rbf_refset.csv", index=False, header=True)

1316
24
